<a href="https://colab.research.google.com/github/denskoy/dolzhenkov_task/blob/main/Final_task_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=b79c31c0edafa5c6e02d4a86f3a97bdf671d9f2475e38360df40993036833847
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [70]:
from os import sep
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType, FloatType, IntegerType, TimestampType
from pyspark.sql.functions import desc, asc, month, count, to_date, sum
spark = SparkSession.builder \
        .appName("films_actors") \
        .getOrCreate()
schema = StructType([
    StructField("ip", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("method", StringType(), True),
    StructField("url", StringType(), True),
    StructField("response_code", StringType(), True),
    StructField("response_size", IntegerType(), True)
])
df = spark.read.csv("/content/web_server_logs.csv", header=True, schema=schema, sep=",")
#Группировка данных по IP и подсчет количества запросов для каждого IP
ip_df = df.groupBy("ip").agg(count("*").alias("request_count")).orderBy(desc("request_count")).limit(10)
print("Top 10 active IP addresses:")
ip_df.show()
#Группировка данны по HTTP-методу и подсчет количества запросов для каждого метода
method_df = df.groupBy("method").agg(count("*").alias("method_count")).orderBy(desc("method_count"))
print("Request count by HTTP method:")
method_df.show()
#Фильтрация и подсчет количества запросов с кодом ответа 404.
code_df = df.filter(df.response_code == "404").agg(count("*"))
print("Number of 404 response codes:" + str(code_df.collect()[0][0]))
#Группировка данных по дате и суммирование размера ответов.
date_df = df.groupBy(to_date("timestamp")).agg(sum("response_size")).orderBy(to_date("timestamp"))
print("Total response size by day:")
date_df.show()
spark.stop

Top 10 active IP addresses:
+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
|  2.230.127.126|            2|
| 65.179.201.149|            1|
|  195.192.23.43|            1|
|121.173.134.203|            1|
|   8.19.141.136|            1|
|    8.149.14.73|            1|
|     72.20.3.30|            1|
|   41.166.44.36|            1|
|   15.79.23.143|            1|
|  84.28.120.239|            1|
+---------------+-------------+

Request count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|  POST|       25214|
|   GET|       25049|
|DELETE|       24982|
|   PUT|       24755|
+------+------------+

Number of 404 response codes:25053
Total response size by day:
+------------------+------------------+
|to_date(timestamp)|sum(response_size)|
+------------------+------------------+
|        2024-01-01|           2059165|
|        2024-01-02|           2025262|
|        2024-01-03|           1906083|
|        2024

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x78649449f640>>